<img src="images/Plane.jpeg" width="800">

# Inserting external data sources into PostgreSQL - Aircrafts
In the last jupyter notebook we learned how to import a csv file directly from the web into Python and write its data into our PostgreSQL database. In this notebook we're going apply similar steps, but this time we're going to add data about aircrafts to our database. Also, instead of importing the data directly from the web, we're going to save it to our local file storage since the files are stored in a zip file. This means we will be unpacking the zip file first, then bring the data into the right shape and format through subsetting, merging and cleaning and ultimately insert it into our PostgreSQL database.  

The aircraft data we're going to work with can be downloaded from the Federal Aviation Administration website. Check out this [link](https://www.faa.gov/licenses_certificates/aircraft_certification/aircraft_registry/releasable_aircraft_download/) to find a description of the data as well as links for downloading either the entire database or yearly data.  

For our use case, we will be downloading the entire database. To do this, we need to import a package that we have worked with previously and is perfect for getting data from the web.  
Complete the code below and import the necessary package.

In [ ]:
# Import package


Next we're going to create the download url and set the download path.

In [ ]:
# Create download URL
zip_file = f'ReleasableAircraft.zip'
url = f'https://registry.faa.gov/database/{zip_file}'

# Set download path
path ='data/'

Now it's time to download the Aircraft Registration Database and save it to our the previoulsy set download path.  
The file is ~60MB, so depending on your internet speed this might take a few seconds.

In [ ]:
# Download the database
r = requests.___(___)

# Save database to local file storage
with open(path+zip_file, 'wb') as f:
    f.write(r.content)

To make sure the file was downloaded successfully, go to the data folder in this repository and check its content. The folder should have a file called ReleasableAircraft.zip.  

The next steps are to unpack the zip file and load the necessary files into Python. Conveniently there is a package to make unpacking a zip file in paython easy, it's called <ins>zipfile</ins>.  

Complete the code below and import the zipfile package.

In [ ]:
# Import zipfile package


Now it's time to unpack the zip file. First, we're going to use the <ins>ZipFile()</ins> function and pass it  the path to the file as its first argument. As the second argument, in order to read an existing file, we pass it the parameter 'r'. To extract the ZipFile object we can either use the <ins>extract()</ins> function, to extract a single file, or the <ins>extractall()</ins> function, to extract all files from the archive.  

Complete the code below and either extract all files or only the MASTER.txt, ACFTREF.txt and ardata.pdf file using the extract() or extractall() function respectively.

In [ ]:
# Unzip zip file


Good job! Now that we have the necessary files it's time to load them into Python. The MASTER.txt contains all registered aircrafts, exactly what we need.  
Complete the code below, import the MASTER.txt file and save it in a variable called master.

In [ ]:
# Import the necessary package


# Read MASTER.txt file and assign to variable master


# Print first 5 rows


Next, let's have a look at the column names.  
Complete the code below and print all column names with their non-null counts and data types.

In [ ]:
# Print master info


In total we have 287554 rows across 33 columns, no null values and a mix of object and integer data types. On top of that all column names are uppercase and seperated by spaces. As analysts we already know that this is not a preferred format. So before we continue let's make all columns lowercase and replace the spaces with underscores.  
Complete the code below and transform all column names from uppercase to lowercase.

In [ ]:
# Make column names lowercase


# Print all column names


Next, we're going to replace spaces with underscores.

In [ ]:
# Replace spaces with underscores


# Print all column names


Perfect, now that the column names are in a consistent and clean format we can move on to working with the data. Even though the MASTER.txt file has 33 columns, we're only going to need 3: 
1. n-number: the tail number of the aircraft
2. mfr_mdl_code: the manufacturer model code
3. year_mfr: the year the aircraft was manufactured

Complete the code below, create a subset of the master data that only includes the above mentioned columns adn reassign it to the master variable.

In [ ]:
# Create subset with n-number, mfr_mdl_code and year_mfg columns


# Print all column names


Let's clean up the column names a bit more.
Complete the code below and rename the columns to: nnum, code and year.

In [ ]:
# Rename columns to nnum, code and year


# Print all column names


While we're at it, we should remove any leading and trailing whitespace in our nnum and code column.  
Complete the code below and remove any whitepsace from the nnum and code column.

In [ ]:
# Remove whitespace from nnum column


# Remove whitespace from code column


Great job so far! Now we have a file that contains a list of all aircrafts, their manufacturer model code and the year it was manufactured. I agree, this is not a lot of information but we're also not done yet. We still haven't looked at the other file: ACFTREF.txt. This file consists of detailed information for each aircraft such as the number of engines, engine type, number of seats etc. Let's import it and have a look!  
Complete the code below and import the ACFTREF.txt file. Save it in a variable called ref and print its first 5 rows.

In [ ]:
# Read ACFTREF.txt file and assign to variable ref


# Print first 5 rows


Just like we did before, let's have a look at the column names, null values and data types.

In [ ]:
# Print table info


This time we have 12 columns, no null values and a mix of object and integer data types. On top of that all column names are uppercase again, but this time they're seperated with a dash. So before we continue let's make all columns lowercase and replace the dashes with underscores.  
Complete the code below and transform all column names from uppercase to lowercase.

In [ ]:
# Make column names lowercase


# Print all column names


Next, we're going to replace spaces with underscores.

In [ ]:
# Replace spaces with underscores


# Print all column names


Now the column names are in a format that makes it a lot easier to work with the table. Next we're going to create a subset of the data since we won't be needing all the columns present in the table.
Complete the code below and create a subset that only contains the following columns: 
1. code: aircraft manufacturer, model and series code
2. mfr: name of the aircraft manufacturer
3. model: name of the aircraft model and series
4. type_acft: the id of the aircraft type
5. type_eng: the id of the engine type
6. no_eng: number of engines on the aircraft
7. no_seats: maximum number of seats in the aircraft
8. speed: the aircraft average cruising speed

Complete the code below and create a subset of the ref data that only includes the above mentioned columns.

In [ ]:
# Select columns to keep


# Print all column names


Great, now we have a subset that only contains the columns we're interested in.  
Next, we're going to combine our master and ref dataset on the code column.  
Complete the code below and inner join the master and ref dataset on the code column.  
Save the combined dataset in a new variable called 'all' and print the dataframe's info.

In [ ]:
# Inner join master and ref and assign to all


# Print info


It worked, awesome! Now we have a combined dataset that consists of detailed aircraft information.  Unfortunately, we're not done yet. The two columns type_acft and type_eng only contain ids and no real information about aircraft and engine type. Fortunately, the Federal Aviation Administration includes a documentation file with the database. The file is called ardata.pdf and contains column descriptions as well as values for codes and ids.  

Let's translate the type_eng column first. Looking into the documentation we find 12 different engine types with ids ranging from 0 to 11. First we should check whether we find these ids in the type_eng column.  
Complete the code below and print a list of all distinct type_eng id values.

In [ ]:
# Print list of distinct type_eng ids


Good, the ids in the type_eng column match with the ones listed in the documentation.  
Next, create a list of engine types, called engine, that includes all engine types listed in the documentation. Make sure to store them in the same order they are listed in the documentation.

In [ ]:
# Create list of engine type names: engine
engine = ["None", 
          "___________", 
          "Turbo-prop", 
          "Turbo-shaft", 
          "__________",
          "Turbo-fan", 
          "Ramjet", 
          "2 Cycle", 
          "4 Cycle", 
          "Unknown", 
          "_________", 
          "Rotary"]

Now we're going to add a new column called engine that has the engine name based on the id in type_eng. 

In [ ]:
# Add engine column and translate type_eng id
all['______'] = [engine[i] for i in all['_______'].tolist()]

# Print first 5 rows


Good job, that was certainly not an easy task. Now that we have the actual engine type names in our dataset we don't need the type_eng column anymore.  
Complete the code below and delete the type_eng column.

In [ ]:
# Delete type_eng column


# Print all column names


Now that we know how to translate an id into its actual value, let's do the same for the type_acft column.  
First, let's have a look at the documentation. There we find 11 engine types with ids ranging from 1 to 9 and two letters H and O. This complicates things, but before we look further into this let's check the type_afct column values for matching ids first.

In [ ]:
# Print list of distinct type_acft ids


Again, the ids are matching, which is good news. The bad news is we have a mix of numerical ids and letters and because of the letters the column is in a string format. The approach we used with the type_eng column will not work with the current format of the column and ids. There is no reason to get demotivated though, this is only but a small hurdle we have to overcome. First, let's change the H and O id in the type_acft column into the numerical ids 10 and 11.  
Complete the code below and replace all H values with the number 10 and all O values with the number 11.

In [ ]:
# Replace H with 10 and O with 11



# Print list of distinct type_eng ids


Now even though all ids are numerical values they are still strings due to the column type being string. We can resolve this by changing the column type to numerical.  
Complete the code below and transform the type_acft column to numerical.

In [ ]:
# Convert type_acft column to numerical


# Print data type of type_acft column



Good job! Now we have consistent and clean type_acft values. Let's move on and just like before create a list called acft and store all aircraft types from the documentation in it. Make sure to store them in the same order they are listed in the documentation.

In [ ]:
acft = ["Glider", 
        "_______", 
        "Blimp/Dirigible", 
        "Fixed wing single engine",
        "Fixed wing multi engine", 
        "_________", 
        "Weight-shift-control",
        "Powered Parachute", 
        "_________",
        'Hybrid Lift',
        'Other']

The last step is to translate the ids in the type_acft column into engine type names.  
Complete the code below and add a new column called type that consists of the correct aircraft type names based on the id in the type_acft column.

In [ ]:
# Add aircraft type and delete type-acft column
all['_____'] = [acft[____] for i in all['_______'].tolist()]

# Print first 5 rows


Good job, that was a tricky one! Now that we have the actual aircraft type names in our dataset we don't need the type_acft column anymore.  
Complete the code below and delete the type_acft column.

In [ ]:
# Delete type_acft column


# Print all column names


Done! We successfully translated the ids to their actual values. Before we write the table into our database, let's make sure we have clean and descriptive column names. The first column we are going to change is the nnum column. Lets have a look at its values before we rename it.  
Complete the code below and print the first 5 rows of the nnum column.

In [ ]:
# Print first 5 rows of nnum column


Something is weird. Aren't the tail numbers in our flights data starting with the letter 'N'? Let's check that just to be sure. In order to do this we need to connect to our database and query the unique tail numbers from the flights table. In order to create a connection we need to create a connection object using the connect() function in combination with the credentails for the PostgreSQL database. We stored this in the sql.py file already. All that's left to do is to enter the credentials and import he connection object conn into this notebook.
Fill in the credentials in the sql.py file, complete the code below and import the conn object from the sql.py file.

In [ ]:
# Import conn from sql.py


Next, query the distinct tail numbers from the flights table and store them in a variable f_planes and print the first 5 rows.

In [ ]:
# Store unique tail numbers in f_planes


# Print first 5 rows of f_planes


Indeed, the tail numbers all start with the letter 'N'. To make sure the nnum column really consists of tail numbers and only the letter 'N' is missing, let's calculate how many matching values we have between the two columns.

In [ ]:
# Count matching values in tail_number and nnum


We have 4506 matches, which is quite a significant number. Therefore, we can assume that in order to match the nnum and the tail_number column, all we need to do is add the letter 'N' in front of each value in the nnum column.  
Complete the code below and create a new column in all called tail_number that consists of the letter 'N' and the nnum values.

In [ ]:
# Create tailnum column


# Print first 5 rows


There we go! Now we have a tail number column that we can join with our flights table later on. Let's change the order of the columns and get rid of the code column since we don't need it anymore.

In [ ]:
# Remove code column, change column order and assign to planes
planes = all[['tail_number', 'year', 'type', 'mfr', 'model', 'engine', 'no_eng', 'no_seats', 'speed']]

As a final data cleaning step give the mfr, no_eng and no_seats column a more descriptive name.  
Complete the code below and change the column names mfr into manufacturer, no_eng into engines and no_seats into seats.

In [ ]:
# Change column names
planes.rename(columns={'___' : '_____', '_____' : '______', '_____' : '_____'}, inplace=True)

# Print all column names


Awesome! We finally have a clean dataset with detailed information about aircrafts. Let's check how many unique aircrafts we have in our dataset.  
Complete the code below and count the unique airplanes in the planes variable.

In [ ]:
# Count unique aircrafts


Wow, almost 290k aircrafts. Previously when we counted the matches between the nnum columnd and tail_number column we only had 4506 matches, which is a lot less than we have from the official source. Since we only need a small subset, let's filter out the remaining ones.  
Complete the code below and create a dataframe called semi_table that has all aircrafts from the planes dataset that have matches in the f_planes dataset.

In [ ]:
# Create dataframe with matching values: semi_table
semi_table = planes.merge(f_planes, how='inner', on='tail_number')

Next, we need to use the list of tail numbers from the semi_table dataset to filter our planes dataset.  
Complete the code below and create a variable in_both that returns TRUE for each tail number from the semi_table that can be found in the tail_number column from the planes dataset and FALSE whenever there is not match.

In [ ]:
# Create a series of bool values: in_both
in_both = planes['_________'].isin(_____________)

What's left is to filter the planes dataset based on the boolean values in the in_both variable.  
Complete the code below and create a new variable called planes_in_both that consists of the planes dataset filtered by the in_both variable.

In [ ]:
# Filter planes and store in planes_in_both
planes_in_both = ____________

# Print planes_in_both


Good job! Instead of a huge dataset with all aircrafts we now have a smaller subset that matches the aircrafts we have in our flights table in our PostregSQL database.  

The last step is to write this table into our database. We already learned how to do this using the sql.py file. The credentials should be filled out since we've already done that in the 'From Web to DB' session.  
Make sure the credentials a set up correctly in the sql.py file and import the engine from the sql.py file.


In [ ]:
# Import engine from sql.py


Next, set the table name variable. This will be name of the table that will be written to the PostgreSQL database.

In [ ]:
# IMPORTANT: Set the table_name variable to 'planes_' + your initials/group number
# Example: planes_pw for Philipp Wendt / planes_1 for group1
table_name = '_________'

The final step is to write the dataset to the database.  
Complete the code below and write the dataset stored in planes_in_both to the PostgreSQL database.

In [ ]:
# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        carriers.to_sql(___________, # Name of SQL table
                        con=_______, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

To check if everything worked try querying the table from the database.

In [ ]:
# Query the new planes table


You made it, congratulations!